In [26]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import os

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu,True)

In [27]:
import polars as pl
import numpy as np
import os
directory = '/app/finalDataSets'
# directory = r'D:\github\localt20'

playerStats = tf.convert_to_tensor(np.load(os.path.join(directory,'playerStats.npy')))
balltoball = tf.convert_to_tensor(pl.read_csv(os.path.join(directory,'balltoball.csv')))
teamStats = tf.convert_to_tensor(pl.read_csv(os.path.join(directory,'team12Stats.csv')))

# Shapes of the data
playerStats_shape = playerStats.shape
teamStats_shape = teamStats.shape
balltoball_shape = balltoball.shape
playerStats_shape,teamStats_shape,balltoball_shape

(TensorShape([1156, 22, 23]),
 TensorShape([1156, 24]),
 TensorShape([267455, 6]))

In [44]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet

# Customize model input layer
cnnModel = models.Sequential([
    layers.Input(shape=(22, 23, 1)),  # Original input shape for player stats
    layers.Resizing(32, 32),          # Resize to fit MobileNet input size
    layers.Conv2D(3, (1, 1), activation='relu'),  # Convert 1 channel to 3 channels
    MobileNet(include_top=False, weights='imagenet', input_shape=(32, 32, 3)),
    layers.GlobalAveragePooling2D(),  # Optional: Pooling for fixed-size vector
    layers.Dense(50, activation='relu')  # Optional: Compress to desired number of features
])

cnnModel.summary()

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing_22 (Resizing)          │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 32, 32, 3)      │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenet_1.00_224 (Functional) │ (None, 1, 1, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 50)             │        51,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,280,120 (12.51 MB)

 Trainable params: 3,258,232 (12.43 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [45]:
cnnModel.predict(tf.reshape(playerStats[0],(1,22,23)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


array([[0.10346881, 0.12140499, 0.06192937, 0.03918223, 0.        ,
        0.12243992, 0.08351867, 0.02032251, 0.11849212, 0.14090192,
        0.        , 0.09542371, 0.01449581, 0.06927989, 0.20319808,
        0.0872468 , 0.        , 0.        , 0.13134104, 0.12927541,
        0.19423707, 0.        , 0.        , 0.15560718, 0.        ,
        0.        , 0.        , 0.        , 0.01044249, 0.10435055,
        0.19142486, 0.        , 0.00999201, 0.        , 0.        ,
        0.        , 0.10410088, 0.13050683, 0.15894303, 0.        ,
        0.14521855, 0.        , 0.01968767, 0.15174405, 0.10733835,
        0.        , 0.        , 0.        , 0.        , 0.03313792]],
      dtype=float32)

In [ ]:
rnnModel = models.Sequential([
    layers.LSTM(64, return_sequences=True),
    layers.LSTM(64),
    layers.Dense(64, activation='relu'),
])
rnnModel.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs  # Skip connection

    # Feed Forward Network
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    return x + res  # Skip connection

def create_transformer_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    # Stack of transformer blocks
    for _ in range(2):  # Number of transformer blocks
        x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=32, dropout=0.1)

    x = layers.GlobalAveragePooling1D(data_format='channels_first')(x)  # Pooling layer
    x = layers.Dense(20, activation="relu")(x)  # Example of a dense layer
    outputs = layers.Dense(1, activation="sigmoid")(x)  # Output layer for win/loss prediction

    return models.Model(inputs, outputs)

# Example usage
input_shape = (22, 23)  # 22 players, 23 stats
model = create_transformer_model(input_shape)
model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_36      │ (None, 22, 23)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 22, 23)    │         46 │ input_layer_36[0… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 22, 23)    │     24,343 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 22, 23)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 22, 23)    │          0 │ dropout_4[0][0],  │
│                     │                   │            │ input_layer_36[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 22, 23)    │         46 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 22, 23)    │        552 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 22, 23)    │          0 │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 22, 23)    │          0 │ dropout_5[0][0],  │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 22, 23)    │         46 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 22, 23)    │     24,343 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 22, 23)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 22, 23)    │          0 │ dropout_7[0][0],  │
│                     │                   │            │ add_3[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 22, 23)    │         46 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 22, 23)    │        552 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 22, 23)    │          0 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 22, 23)    │          0 │ dropout_8[0][0],  │
│                     │                   │            │ add_4[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 22)        │          0 │ add_5[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 20)        │        460 │ global_average_p

 Total params: 50,455 (197.09 KB)

 Trainable params: 50,455 (197.09 KB)

 Non-trainable params: 0 (0.00 B)